https://keras.io/getting_started/intro_to_keras_for_engineers/

In [1]:
import numpy as np
import tensorflow as tf
import keras

# Data loading & preprocessing

## Data loading
Keras models accept three types of inputs:

- **NumPy arrays**, just like Scikit-Learn and many other Python-based libraries. This is a good option if your data fits in memory.
- **TensorFlow Dataset objects**. This is a high-performance option that is more suitable for datasets that do not fit in memory and that are streamed from disk or from a distributed filesystem.
- **Python generators** that yield batches of data (such as custom subclasses of the keras.utils.Sequence class).

## Data preprocessing with Keras

This can mean:

- **Tokenization** of string data, followed by token indexing.
- **Feature normalization**.
- **Rescaling** the data to small values (in general, input values to a neural network should be close to zero -- typically we expect either **data with zero-mean** and **unit-variance**, or data **in the [0, 1] range**.

In Keras, you do in-model data preprocessing via preprocessing layers. This includes:

- Vectorizing raw strings of text via the **TextVectorization** layer
- Feature normalization via the **Normalization** layer
- Image rescaling, cropping, or image data augmentation

In [2]:
# Example: turning strings into sequences of integer word indices

from tensorflow.keras.layers import TextVectorization

# Example training data, of dtype `string`.
training_data = np.array([["This is the 1st sample."], ["And here's the 2nd sample."]])

# Create a TextVectorization layer instance. It can be configured to either
# return integer token indices, or a dense token representation (e.g. multi-hot
# or TF-IDF). The text standardization and text splitting algorithms are fully
# configurable.
vectorizer = TextVectorization(output_mode="int")

# Calling `adapt` on an array or dataset makes the layer generate a vocabulary
# index for the data, which can then be reused when seeing new data.
vectorizer.adapt(training_data)

# After calling adapt, the layer is able to encode any n-gram it has seen before
# in the `adapt()` data. Unknown n-grams are encoded via an "out-of-vocabulary"
# token.
integer_data = vectorizer(training_data)
print(integer_data)
print(vectorizer.get_vocabulary())
frase = vectorizer.get_vocabulary()
for palabra in frase :
    print(palabra)

tf.Tensor(
[[4 5 2 9 3]
 [7 6 2 8 3]], shape=(2, 5), dtype=int64)
['', '[UNK]', 'the', 'sample', 'this', 'is', 'heres', 'and', '2nd', '1st']

[UNK]
the
sample
this
is
heres
and
2nd
1st


In [3]:
#  Example: turning strings into sequences of one-hot encoded bigrams

from tensorflow.keras.layers import TextVectorization

# Example training data, of dtype `string`.
training_data = np.array([["This is the 1st sample."], ["And here's the 2nd sample."]])

# Create a TextVectorization layer instance. It can be configured to either
# return integer token indices, or a dense token representation (e.g. multi-hot
# or TF-IDF). The text standardization and text splitting algorithms are fully
# configurable.
vectorizer = TextVectorization(output_mode="binary", ngrams=2)

# Calling `adapt` on an array or dataset makes the layer generate a vocabulary
# index for the data, which can then be reused when seeing new data.
vectorizer.adapt(training_data)

# After calling adapt, the layer is able to encode any n-gram it has seen before
# in the `adapt()` data. Unknown n-grams are encoded via an "out-of-vocabulary"
# token.
integer_data = vectorizer(training_data)
print(integer_data)
print(vectorizer.get_vocabulary())
frase = vectorizer.get_vocabulary()
for palabra in frase :
    print(palabra)

tf.Tensor(
[[0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]], shape=(2, 17), dtype=float32)
['[UNK]', 'the', 'sample', 'this is', 'this', 'the 2nd', 'the 1st', 'is the', 'is', 'heres the', 'heres', 'and heres', 'and', '2nd sample', '2nd', '1st sample', '1st']
[UNK]
the
sample
this is
this
the 2nd
the 1st
is the
is
heres the
heres
and heres
and
2nd sample
2nd
1st sample
1st


In [4]:
# Example: normalizing features

from tensorflow.keras.layers import Normalization

# Example image data, with values in the [0, 255] range
training_data = np.random.randint(0, 256, size=(64, 5, 8, 3)).astype("float32")
print(len(training_data))
print(training_data[0].shape)

normalizer = Normalization(axis=-1)
normalizer.adapt(training_data)

normalized_data = normalizer(training_data)
print("var: %.4f" % np.var(normalized_data))
print("mean: %.4f" % np.mean(normalized_data))
print(normalized_data[0])

64
(5, 8, 3)
var: 1.0000
mean: -0.0000
tf.Tensor(
[[[ 0.08360337 -1.0809995   1.0775554 ]
  [-0.41572946 -0.7288941  -1.3670094 ]
  [-1.3064313   0.02948662 -1.5028186 ]
  [ 1.2442149  -1.3789347   0.50715697]
  [-0.9150623   1.2753979   0.23553862]
  [ 0.2185582   1.5462481   1.2269455 ]
  [-0.5641798   0.13782673  0.39850962]
  [ 0.2185582  -0.34970376  1.4985638 ]]

 [[ 1.52762     1.4920781  -1.434914  ]
  [-1.3469177   1.3701954   0.42567146]
  [ 1.136251    0.48993206  0.11331038]
  [ 0.87983686 -0.7559791   0.22195771]
  [ 1.3386832   0.96392006 -0.9052583 ]
  [ 1.2172239   0.7336973  -0.15830794]
  [-0.9960352  -1.1622546   0.24911954]
  [-0.05135145  0.6524422  -0.17188886]]

 [[ 0.3670085  -0.16010857  0.5750615 ]
  [ 0.40749496  0.78786737 -0.17188886]
  [ 0.25904465  0.5847297  -1.6250468 ]
  [-1.0635127  -1.56853    -1.1361339 ]
  [ 0.9878007   1.410823   -0.41634533]
  [ 0.17807175  0.8555799  -0.93242013]
  [-0.5776753   1.7087582  -1.0818102 ]
  [-0.55068433 -0.5528414 

# Building models with the Keras Functional API

A "layer" is a **simple input-output transformation** (such as the scaling & center-cropping transformations above). For instance, here's a linear projection layer that maps its inputs to a **16-dimensional feature space:**
```python
tf.keras.layers.Dense(
    units,
    activation=None,
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs
)
```


In [5]:
dense = keras.layers.Dense(units=16)
print(type(dense))
# print(dense.output_shape)
# AttributeError: The layer "dense_6" has never been called and thus has no defined output shape.

<class 'keras.src.layers.core.dense.Dense'>


**Start** by **specifying** the shape (and optionally the dtype) of **your inputs**. If any dimension of your input can vary, you can specify it as None. For instance, an input for 200x200 RGB image would have shape (200, 200, 3), but an input for RGB images of any size would have shape (None, None, 3).

In [6]:
# Let's say we expect our inputs to be RGB images of arbitrary size
inputs = keras.Input(shape=(None, None, 3))

After defining your input(s), you can **chain layer transformations** on top of your inputs, **until** your final **output**:

In [7]:
from tensorflow.keras.layers import CenterCrop
from tensorflow.keras.layers import Rescaling

from tensorflow.keras import layers

# Center-crop images to 150x150
x = CenterCrop(height=150, width=150)(inputs)
# Rescale images to [0, 1]
x = Rescaling(scale=1.0 / 255)(x)

# Apply some convolution and pooling layers
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

# Apply global average pooling to get flat feature vectors
x = layers.GlobalAveragePooling2D()(x)

# Add a dense classifier on top
num_classes = 10
outputs = layers.Dense(num_classes, activation="softmax")(x)

Instantiate a **Model object**:<br>
This model behaves basically like a bigger layer

In [8]:
model = keras.Model(inputs=inputs, outputs=outputs)

You can call it on batches of data, like this:

In [9]:
data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")
processed_data = model(data)
print(processed_data.shape)
print(type(processed_data))
print(processed_data)

2023-08-12 09:58:34.470092: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 179437568 exceeds 10% of free system memory.
2023-08-12 09:58:35.294730: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 179437568 exceeds 10% of free system memory.
2023-08-12 09:58:35.851387: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 179437568 exceeds 10% of free system memory.


(64, 10)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[0.07410397 0.11647474 0.10202663 0.08362741 0.11587635 0.09549182
  0.11877757 0.0856298  0.11563359 0.09235824]
 [0.07372666 0.11657809 0.10250198 0.08331922 0.11591654 0.0955729
  0.11870168 0.08560199 0.1153931  0.09268787]
 [0.07431979 0.11635533 0.10200357 0.08408103 0.11571723 0.09521581
  0.11849707 0.08568646 0.11562957 0.09249421]
 [0.0739358  0.1167133  0.10225845 0.08339628 0.11575212 0.09559432
  0.11906067 0.08545051 0.1155792  0.09225936]
 [0.07413264 0.11663312 0.10225221 0.08366787 0.1163573  0.0954052
  0.1187713  0.08530771 0.11508706 0.09238551]
 [0.07409769 0.1164202  0.1020803  0.08357675 0.11646608 0.09532275
  0.11825904 0.08567231 0.11578619 0.0923186 ]
 [0.07376501 0.11683293 0.10237014 0.08373573 0.1155677  0.095287
  0.11852095 0.08582567 0.11570112 0.09239374]
 [0.07405906 0.11665686 0.10199686 0.08363652 0.11624402 0.09526346
  0.11848207 0.08567549 0.11570782 0.09227791]
 [0.074004

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 center_crop (CenterCrop)    (None, 150, 150, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 150, 150, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        9248      
                                                             

# Training models with fit()

**Before** you can call **fit()**, you **need** to specify **an optimizer and a loss function** (we assume you are already familiar with these concepts). This is the **compile()** step:



In [11]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())
# que es lo mismo que 
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

you can start **"fitting"** the model to the data. Here's what fitting a model looks like with NumPy data:
```python
model.fit(numpy_array_of_samples, numpy_array_of_labels,
          batch_size=32, epochs=10)

```


**Besides the data**, you have to specify **two key parameters**: the **batch_size** and the number of **epochs** (iterations on the data). Here our data will get sliced on **batches of 32 samples**, and the model will **iterate 10 times over the data during training**.

Here's what fitting a model looks like with a dataset:
```python
model.fit(dataset_of_samples_and_labels, epochs=10)
```

# LET's play with a TOY example

In [20]:
# Get the data as Numpy arrays
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Build a simple model
inputs = keras.Input(shape=(28, 28))
x = layers.Rescaling(1.0 / 255)(inputs)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics ='accuracy')

# Train the model for 1 epoch from Numpy data
batch_size = 64
print("Fit on NumPy data")
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=1)

# Train the model for 1 epoch using a dataset
# y utiliza tensorboard para visualizar
import tensorflow as tf
import datetime

callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs')
]

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
print("Fit on Dataset")
history = model.fit(dataset, epochs=10, callbacks=callbacks)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 rescaling_2 (Rescaling)     (None, 28, 28)            0         
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               100480    
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118282 (462.04 KB)
Trainable params: 118282 (

In [19]:
print(history.history)
print(type(history))


{'loss': [0.11234305053949356], 'accuracy': [0.9666666388511658]}
<class 'keras.src.callbacks.History'>


### Keeping track of performance metrics
#### Monitoring metrics
#### Passing validation data to fit()
#### Using callbacks for checkpointing (and more)
#### Monitoring training progress with TensorBoard


## After fit(): evaluate test performance & make predictions on new data
cuidado esto est mal. Tiene que buscar un dataset de validación

In [14]:
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
loss, acc = model.evaluate(dataset)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

938/938 [==============================] - 8s 8ms/step - loss: 0.0869 - accuracy: 0.9725
loss: 0.09
acc: 0.97


In [15]:
predictions = model.predict(val_dataset)
print(predictions.shape)

157/157 [==============================] - 1s 8ms/step
(10000, 10)
